### SQL Assignment – Maven Movies Database

---

#### **1. Create Table with Constraints**

```sql
CREATE TABLE employees (
  emp_id INT PRIMARY KEY NOT NULL,
  emp_name TEXT NOT NULL,
  age INT CHECK (age >= 18),
  email TEXT UNIQUE,
  salary DECIMAL DEFAULT 30000
);
```

---

#### **2. Purpose of Constraints**

Constraints enforce rules at the table level to maintain data integrity.
Examples:

* **PRIMARY KEY**: Uniquely identifies rows (e.g., emp\_id)
* **NOT NULL**: Prevents NULL values
* **CHECK**: Enforces a condition (e.g., age >= 18)
* **UNIQUE**: Prevents duplicates (e.g., email)
* **FOREIGN KEY**: Maintains referential integrity

---

#### **3. NOT NULL & Primary Key**

* `NOT NULL` ensures a field always has a value.
* A primary key **cannot** contain NULLs because it must uniquely identify every record.

---

#### **4. Add/Remove Constraints**

**Add Constraint**:

```sql
ALTER TABLE employees ADD CONSTRAINT chk_age CHECK (age >= 18);
```

**Remove Constraint**:

```sql
ALTER TABLE employees DROP CONSTRAINT chk_age;
```

---

#### **5. Violating Constraints**

If a constraint is violated:

```sql
INSERT INTO employees (emp_id, emp_name, age) VALUES (1, 'John', 15);
```

**Error**: CHECK constraint failed: age >= 18

---

#### **6. Add Constraints to Existing Table**

```sql
ALTER TABLE products ADD CONSTRAINT pk_product PRIMARY KEY (product_id);
ALTER TABLE products ALTER COLUMN price SET DEFAULT 50.00;
```

---

#### **7. INNER JOIN - Students and Classes**

```sql
SELECT student_name, class_name
FROM students
INNER JOIN classes ON students.class_id = classes.class_id;
```

---

#### **8. Orders and Products (LEFT JOIN)**

```sql
SELECT o.order_id, c.customer_name, p.product_name
FROM products p
LEFT JOIN orders o ON p.product_id = o.product_id
LEFT JOIN customers c ON o.customer_id = c.customer_id;
```

---

#### **9. Total Sales Amount Per Product**

```sql
SELECT p.product_name, SUM(oi.quantity * oi.price) AS total_sales
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_name;
```

---

#### **10. Quantity per Customer per Order**

```sql
SELECT o.order_id, c.customer_name, oi.quantity
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id;
```

---

### SQL COMMANDS (Maven Movies)

1. Primary Key: `film_id` in `film`; Foreign Key: `category_id` in `film_category`
2. `SELECT * FROM actor;`
3. `SELECT * FROM customer;`
4. `SELECT DISTINCT country FROM country;`
5. `SELECT * FROM customer WHERE active = 1;`
6. `SELECT rental_id FROM rental WHERE customer_id = 1;`
7. `SELECT * FROM film WHERE rental_duration > 5;`
8. `SELECT COUNT(*) FROM film WHERE replacement_cost BETWEEN 15 AND 20;`
9. `SELECT COUNT(DISTINCT first_name) FROM actor;`
10. `SELECT * FROM customer LIMIT 10;`
11. `SELECT * FROM customer WHERE first_name LIKE 'b%' LIMIT 3;`
12. `SELECT title FROM film WHERE rating = 'G' LIMIT 5;`
13. `SELECT * FROM customer WHERE first_name LIKE 'a%';`
14. `SELECT * FROM customer WHERE first_name LIKE '%a';`
15. `SELECT city FROM city WHERE city LIKE 'a%a' LIMIT 4;`
16. `SELECT * FROM customer WHERE first_name LIKE '%NI%';`
17. `SELECT * FROM customer WHERE first_name LIKE '_r%';`
18. `SELECT * FROM customer WHERE first_name LIKE 'a____%';`
19. `SELECT * FROM customer WHERE first_name LIKE 'a%o';`
20. `SELECT * FROM film WHERE rating IN ('PG', 'PG-13');`
21. `SELECT * FROM film WHERE length BETWEEN 50 AND 100;`
22. `SELECT * FROM actor LIMIT 50;`
23. `SELECT DISTINCT film_id FROM inventory;`

---

### Functions – Maven Movies

1. `SELECT COUNT(*) FROM rental;`
2. `SELECT AVG(rental_duration) FROM film;`
3. `SELECT UPPER(first_name), UPPER(last_name) FROM customer;`
4. `SELECT rental_id, MONTH(rental_date) FROM rental;`
5. `SELECT customer_id, COUNT(*) FROM rental GROUP BY customer_id;`
6. `SELECT store_id, SUM(amount) FROM payment GROUP BY store_id;`
7. Join `film_category`, `film`, and `rental`:

```sql
SELECT fc.category_id, COUNT(*)
FROM film_category fc
JOIN film f ON fc.film_id = f.film_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY fc.category_id;
```

8. `SELECT l.name, AVG(f.rental_rate) FROM film f JOIN language l ON f.language_id = l.language_id GROUP BY l.name;`

---

### Joins

9. Movie rented by customer:

```sql
SELECT f.title, c.first_name, c.last_name
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN customer c ON r.customer_id = c.customer_id;
```

10. Actors in “Gone with the Wind”:

```sql
SELECT a.first_name, a.last_name
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE f.title = 'Gone with the Wind';
```

11. Total amount spent per customer:

```sql
SELECT c.first_name, c.last_name, SUM(p.amount)
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id;
```

12. Movies rented by customers in 'London':

```sql
SELECT c.first_name, c.last_name, f.title
FROM customer c
JOIN address a ON c.address_id = a.address_id
JOIN city ci ON a.city_id = ci.city_id
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE ci.city = 'London';
```

---

### Advanced Joins / GROUP BY / Windows

13. Top 5 rented movies:

```sql
SELECT f.title, COUNT(*) AS rental_count
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.title
ORDER BY rental_count DESC
LIMIT 5;
```

14. Customers renting from both stores:

```sql
SELECT customer_id
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
GROUP BY customer_id
HAVING COUNT(DISTINCT i.store_id) = 2;
```

---

### Windows Functions

1. Rank customers by amount:

```sql
SELECT customer_id, SUM(amount), RANK() OVER (ORDER BY SUM(amount) DESC)
FROM payment GROUP BY customer_id;
```

2. Cumulative film revenue:

```sql
SELECT film_id, rental_date, SUM(amount) OVER (PARTITION BY film_id ORDER BY rental_date) AS cumulative_revenue
FROM payment
JOIN rental ON payment.rental_id = rental.rental_id
JOIN inventory ON rental.inventory_id = inventory.inventory_id;
```

3. Avg rental by similar lengths:

```sql
SELECT film_id, length, AVG(rental_duration) OVER (PARTITION BY length) AS avg_rental_duration
FROM film;
```

4. Top 3 films by rental count per category:

```sql
WITH FilmRank AS (
  SELECT fc.category_id, f.title, COUNT(*) AS rentals,
         RANK() OVER (PARTITION BY fc.category_id ORDER BY COUNT(*) DESC) AS rank
  FROM film_category fc
  JOIN film f ON fc.film_id = f.film_id
  JOIN inventory i ON f.film_id = i.film_id
  JOIN rental r ON i.inventory_id = r.inventory_id
  GROUP BY fc.category_id, f.title
)
SELECT * FROM FilmRank WHERE rank <= 3;
```

5. Rental count vs avg:

```sql
WITH AvgRentals AS (
  SELECT customer_id, COUNT(*) AS rental_count FROM rental GROUP BY customer_id
),
OverallAvg AS (
  SELECT AVG(rental_count) AS avg_count FROM AvgRentals
)
SELECT * FROM AvgRentals WHERE rental_count > (SELECT avg_count FROM OverallAvg);
```

---

### Normalization & CTEs

1. **1NF**: Split repeating groups into separate rows. Example: separate phone numbers column.
2. **2NF**: Remove partial dependencies. Split tables using composite PKs.
3. **3NF**: Remove transitive dependencies. Ensure non-key attributes depend only on PK.
4. **Normalization Process**: Start by identifying repeating groups and then use stepwise decomposition.
5. **CTE Actor Films**:

```sql
WITH ActorFilmCount AS (
  SELECT actor_id, COUNT(*) AS films FROM film_actor GROUP BY actor_id
)
SELECT a.first_name, a.last_name, af.films
FROM ActorFilmCount af
JOIN actor a ON a.actor_id = af.actor_id;
```

6. **CTE Film + Language**:

```sql
WITH FilmLang AS (
  SELECT f.title, l.name AS language, f.rental_rate
  FROM film f JOIN language l ON f.language_id = l.language_id
)
SELECT * FROM FilmLang;
```

7. **CTE Customer Revenue**:

```sql
WITH CustomerPayments AS (
  SELECT customer_id, SUM(amount) AS total FROM payment GROUP BY customer_id
)
SELECT * FROM CustomerPayments;
```

8. **CTE Rank by Duration**:

```sql
WITH FilmRanks AS (
  SELECT title, RANK() OVER (ORDER BY rental_duration DESC) AS duration_rank FROM film
)
SELECT * FROM FilmRanks;
```

9. **CTE > 2 Rentals**:

```sql
WITH FrequentCustomers AS (
  SELECT customer_id FROM rental GROUP BY customer_id HAVING COUNT(*) > 2
)
SELECT * FROM customer WHERE customer_id IN (SELECT customer_id FROM FrequentCustomers);
```

10. **CTE Monthly Rentals**:

```sql
WITH MonthlyRentals AS (
  SELECT MONTH(rental_date) AS month, COUNT(*) AS rentals FROM rental GROUP BY MONTH(rental_date)
)
SELECT * FROM MonthlyRentals;
```

11. **CTE Actor Pairs**:

```sql
WITH ActorPairs AS (
  SELECT fa1.actor_id AS actor1, fa2.actor_id AS actor2, fa1.film_id
  FROM film_actor fa1
  JOIN film_actor fa2 ON fa1.film_id = fa2.film_id AND fa1.actor_id < fa2.actor_id
)
SELECT a1.first_name || ' ' || a1.last_name AS Actor_1,
       a2.first_name || ' ' || a2.last_name AS Actor_2,
       film_id
FROM ActorPairs
JOIN actor a1 ON a1.actor_id = actor1
JOIN actor a2 ON a2.actor_id = actor2;
```

12. **Recursive CTE: Employees Under Manager**:

```sql
WITH RECURSIVE Subordinates AS (
  SELECT staff_id, first_name, last_name, reports_to
  FROM staff WHERE reports_to = 1
  UNION ALL
  SELECT s.staff_id, s.first_name, s.last_name, s.reports_to
  FROM staff s
  JOIN Subordinates sub ON s.reports_to = sub.staff_id
)
SELECT * FROM Subordinates;
```

---

*End of SQL Assignment Document*
